In [1]:
from model import Posicao, Extrato, AwsModel
from views import FundoInvestimento, Acao, FundoImobiliario
#from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime
import os

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [33]:
#print(os.environ.get('$AWS_ACCESS_KEY_ID'))
os.environ['TESTE_CHAVE_GABRIEL'] = 'username'


os.getenv('TESTE_CHAVE_GABRIEL')

'username'

In [21]:
aws_model = AwsModel()
aws_model.load_data_s3()

# model
posicao = Posicao(aws_model.df_list_pos)
extrato = Extrato(2010, 2020, aws_model.extrato, aws_model.extrato_bolsa)

# view
fundo_investimento = FundoInvestimento(
    posicao=posicao.fis, 
    extrato=extrato.extrato_fis
)
acoes = Acao(posicao=posicao.acoes, extrato=extrato.extrato_acoes)
fiis = FundoImobiliario(posicao=posicao.fiis, extrato=extrato.extrato_fiis)

WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file siz

/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


**Mapeado todos os FIs**

Verifica se existe FI no extrato não mapeado

In [6]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
#extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

## Analise FIs

In [29]:
resumo_fi = fundo_investimento.resumo

In [34]:
print('Resumo por Fi:')

tmp = resumo_fi[resumo_fi['data_posicao'] <= '2020-12-31']\
        .groupby(['Nome'])\
        .agg(posicao=('Total Bruto', 'last'), 
             total_aporte=('aporte', 'sum'), 
             total_rendimento=('rendimento', 'sum'), 
             total_retirada=('retirada', 'sum')).reset_index()

tmp.sort_values(['total_rendimento'], ascending=False)

Resumo por Fi:


,Nome,posicao,total_aporte,total_rendimento,total_retirada
3,Bahia AM Maraú FIC de FIM,"85,897.97","73,000.00","12,897.97",0.00
6,Mauá Macro FIC FIM,"28,882.28","23,500.00","5,382.28",0.00
11,XP Long Short FIC FIM,0.00,"15,000.00","4,822.42",0.00
0,AZ Quest Multi FIC FIM,0.00,"15,000.00","3,591.84","18,591.84"
9,Polo Norte I FIC FIM,"33,556.88","30,000.00","3,556.88",0.00
7,Mirae Asset Multimercado,0.00,"16,000.00","3,122.71","19,122.71"
1,Azul Quantitativo FIM,0.00,"15,200.00","2,624.47","17,824.47"
8,NP Hedge FIC FIM,0.00,"5,000.00",144.33,"5,144.33"
10,XP Absoluto Consumo FIA,0.00,"2,400.00",-93.26,"2,306.74"
2,BNP Paribas Inflação FI RF,0.00,"10,300.00",-131.86,"10,168.14"


## Analise Ações

In [35]:
resumo_acoes = acoes.calcula_resumo(2010, 2020)

In [36]:
resumo_papel = resumo_acoes[resumo_acoes['Data'] <= '2020-12-31']\
        .groupby(['Papel'])\
        .agg(total_financeiro=('Financeiro', 'last'), 
            total_aporte=('aporte', 'sum'),
            total_retirada=('retirada', 'sum'),
            total_rendimento=('rendimento', 'sum')).reset_index()

resumo_papel['%'] = resumo_papel['total_rendimento'] / resumo_papel['total_aporte'] * 100

resumo_papel.sort_values('total_rendimento', ascending=False)

,Papel,total_financeiro,total_aporte,total_retirada,total_rendimento,%
5,MGLU3,"74,816.00","27,600.00",0.00,"47,216.00",171.07
8,WEGE3,"22,071.00","10,095.00",0.00,"11,976.00",118.63
0,BPAC11,"2,059.72","7,190.00","15,500.00","10,369.72",144.22
6,PETR4,"9,960.00","5,444.00",0.00,"4,516.00",82.95
3,ITSA4,"25,344.00","22,505.00",0.00,"2,839.00",12.61
7,TSLA34,"4,772.50","3,622.50",0.00,"1,150.00",31.75
2,EGIE3,"8,468.00","9,068.00",0.00,-600.00,-6.62
4,JHSF3,"4,404.00","12,944.00","7,300.00","-1,240.00",-9.58
1,COGN3,"5,628.00","7,044.00",0.00,"-1,416.00",-20.10


## FIIs

In [37]:
resumo_fiis = fiis.calcula_resumo(2010, 2020)

In [38]:
resfiis_papel = resumo_fiis[resumo_fiis['Data'] <= '2020-12-31']\
        .groupby(['Papel'])\
        .agg(total_financeiro=('Financeiro', 'last'), 
            total_aporte=('aporte', 'sum'),
            total_retirada=('retirada', 'sum'),
            total_rendimento=('rendimento', 'sum')).reset_index()

resfiis_papel['%'] = resfiis_papel['total_rendimento'] / resfiis_papel['total_aporte'] * 100

resfiis_papel.sort_values('total_rendimento', ascending=False)

,Papel,total_financeiro,total_aporte,total_retirada,total_rendimento,%
0,BCFF11,"31,838.76","31,025.60",0.00,813.16,2.62
2,HGLG11,"2,503.80","2,330.55",0.00,173.25,7.43
6,TBOF11,0.00,"5,200.00","5,199.00",-1.00,-0.02
1,CNES11,712.36,885.17,0.00,-172.81,-19.52
5,RNGO11,"3,483.90","4,110.61",0.00,-626.71,-15.25
4,OULG11,"4,443.00","5,677.50",0.00,"-1,234.50",-21.74
3,IBFF11,"3,850.00","5,155.00",0.00,"-1,305.00",-25.32


## Grafico de rendimento

In [77]:
df1 = resumo_acoes[resumo_acoes['Data'] <= '2020-12-31'].groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()

In [78]:
df2 = resumo_fiis[resumo_fiis['Data'] <= '2020-12-31'].groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()

In [79]:
df3 = resumo_fi[resumo_fi['data_posicao'] <= '2020-12-31'].groupby(['data_posicao', 'ano', 'mes'])\
        .agg(financeiro=('Total Bruto', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'data_posicao': 'Data'})

In [171]:
df_graph1 = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)

df_graph1 = df_graph1.groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index().fillna(0)

df_graph1['%'] = df_graph1['rendimento'] / df_graph1['financeiro'] * 100
df_graph1['renda_acum'] = df_graph1['rendimento'].cumsum()
df_graph1['aporte_acum'] = df_graph1['aporte'].cumsum()
df_graph1['retirada_acum'] = df_graph1['retirada'].cumsum()
df_graph1['investido'] = df_graph1['aporte_acum'] - df_graph1['retirada_acum']
df_graph1['% renda_acum'] = df_graph1['rendimento'].cumsum() / (df_graph1['investido']) * 100

In [172]:
df_graph1.fillna(0)

,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
0,2010-01-31,2010,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2010-02-28,2010,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2010-03-31,2010,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2010-04-30,2010,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2010-05-31,2010,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,2010-06-30,2010,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,2010-07-31,2010,7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,2010-08-31,2010,8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2010-09-30,2010,9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,2010-10-31,2010,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [173]:
df_graph1.tail(1)

,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
130,2020-12-31,2020,11,"371,436.03","3,622.50",0.00,"11,691.12",3.15,"96,937.58","446,396.93","162,076.06","284,320.87",34.09


In [229]:
def create_graph1(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
                y=df['%'],
                mode='lines',
                name='% Rendimento',
                textposition='top center',
                text=df['%'].apply(lambda x: f'{x:,.2f}%'),
                marker=dict(size=7),
                line=dict(color='rgb(115,115,115)', width=1.8),
                opacity=.8))  

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()

In [230]:
create_graph1(df_graph1[(df_graph1['Data'] >= '2014-01-01')].sort_values('Data'))

In [212]:
def create_graph3(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
                y=df['% renda_acum'],
                mode='lines',
                name='% Rendimento',
                textposition='top center',
                text=df['% renda_acum'].apply(lambda x: f'{x:,.2f}%'),
                marker=dict(size=7),
                line=dict(color='rgb(115,115,115)', width=1.8),
                opacity=.8))

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()
    
create_graph3(df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data'))

In [231]:
def create_graph2(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
            y=df['renda_acum'],
            mode='lines',
            name = 'Rendimento Acumulado',
            text=df['renda_acum'].apply(lambda x: f'{x/1000:,.0f}K'),
            #textposition='outside',
            line=dict(color='rgb(115,115,115)'), 
            line_shape='linear'
            ))            
    
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()

In [232]:
create_graph2(df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data'))

In [215]:
import plotly.express as px

df_tmp = df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data')

fig = px.line(df_tmp, x='Data', y='renda_acum', title='Time Series with Range Slider and Selectors')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()